<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Bot con NLTK utilizando un corpus de wikipedia


In [1]:
import json
import string
import random
import re
import urllib.request

import numpy as np
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Datos
Se consumira los datos del artículo de wikipedia de la obra literaria de la Tierra Media creada por J. R. R. Tolkien.

In [2]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Tierra_Media')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [3]:
# Demos un vistazo
article_text

'la tierra media es un continente ficticio en el que transcurren la mayor parte de las historias que el autor británico j. r. r. tolkien escribió para su legendarium. la novela el silmarillion narra la lucha que mantienen por el control del mundo, y de la tierra media, los valar, los elfos y los hombres contra morgoth y sus siervos. en épocas posteriores, después de la derrota de morgoth y de su expulsión del mundo, su papel es continuado por su lugarteniente, sauron, titular y principal antagonista de la novela el señor de los anillos. los valar dejan de participar de forma directa en los asuntos de la tierra media tras la derrota de morgoth, pero tiempo después envían a los istari para ayudar en la lucha contra sauron; los más importantes son gandalf el gris, quien se mantiene fiel a su misión y resulta crucial en la lucha para derrotar a sauron, y saruman el blanco, quien sin embargo se corrompe y se alía con el enemigo para apoderarse de la tierra media. otras razas que participan 

In [4]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 42410


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [5]:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [6]:
# Demos un vistazo
text

'la tierra media es un continente ficticio en el que transcurren la mayor parte de las historias que el autor británico j. r. r. tolkien escribió para su legendarium. la novela el silmarillion narra la lucha que mantienen por el control del mundo, y de la tierra media, los valar, los elfos y los hombres contra morgoth y sus siervos. en épocas posteriores, después de la derrota de morgoth y de su expulsión del mundo, su papel es continuado por su lugarteniente, sauron, titular y principal antagonista de la novela el señor de los anillos. los valar dejan de participar de forma directa en los asuntos de la tierra media tras la derrota de morgoth, pero tiempo después envían a los istari para ayudar en la lucha contra sauron; los más importantes son gandalf el gris, quien se mantiene fiel a su misión y resulta crucial en la lucha para derrotar a sauron, y saruman el blanco, quien sin embargo se corrompe y se alía con el enemigo para apoderarse de la tierra media. otras razas que participan 

In [7]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 42364


### 3 - Dividir el texto en sentencias y en palabras

In [8]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [9]:
# Demos un vistazo
corpus[:10]

['la tierra media es un continente ficticio en el que transcurren la mayor parte de las historias que el autor británico j. r. r. tolkien escribió para su legendarium.',
 'la novela el silmarillion narra la lucha que mantienen por el control del mundo, y de la tierra media, los valar, los elfos y los hombres contra morgoth y sus siervos.',
 'en épocas posteriores, después de la derrota de morgoth y de su expulsión del mundo, su papel es continuado por su lugarteniente, sauron, titular y principal antagonista de la novela el señor de los anillos.',
 'los valar dejan de participar de forma directa en los asuntos de la tierra media tras la derrota de morgoth, pero tiempo después envían a los istari para ayudar en la lucha contra sauron; los más importantes son gandalf el gris, quien se mantiene fiel a su misión y resulta crucial en la lucha para derrotar a sauron, y saruman el blanco, quien sin embargo se corrompe y se alía con el enemigo para apoderarse de la tierra media.',
 'otras raza

In [10]:
# Demos un vistazo
words[:20]

['la',
 'tierra',
 'media',
 'es',
 'un',
 'continente',
 'ficticio',
 'en',
 'el',
 'que',
 'transcurren',
 'la',
 'mayor',
 'parte',
 'de',
 'las',
 'historias',
 'que',
 'el',
 'autor']

In [11]:
print("Vocabulario:", len(words))

Vocabulario: 8102


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [12]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus de wikipedia

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias ensayar:
- Gandalf
- Tolkien
- Mal
- Ainur

(A iluvatar no le saco el acento, si colocas Iluvatar no reconoce pero si colocas Ilúvatar si)

In [14]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

     |████████████████████████████████| 3.4 MB 30.3 MB/s 
     |████████████████████████████████| 206 kB 43.6 MB/s 
     |████████████████████████████████| 1.9 MB 35.6 MB/s 
     |████████████████████████████████| 961 kB 32.8 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 3.0 MB 26.6 MB/s 


In [16]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://16425.gradio.app
Interface loading below...


Ilubatar


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Iluvatar


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...

ainur


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


ilúvatar


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


KeyboardInterrupt: ignored